In [6]:
import sys
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [7]:
spark = SparkSession.builder.appName('TF-IDF').getOrCreate()
sc = spark.sparkContext

In [12]:
def split_file(x):
    value=x.split('\t')
    return (value[0], value[1].split(' '))

In [16]:
data = (sc.textFile('cleaned.txt')
    .map(lambda x: split_file(x))
    .toDF(['id', 'words'])
)

In [18]:
hashingTF = HashingTF(inputCol="words", outputCol='features', numFeatures=1000)
tf = hashingTF.transform(data)

In [19]:
idf = IDF(inputCol='features', outputCol='idf')
model = idf.fit(tf)
tf_idf = model.transform(tf)